In [1]:
import pandas as pd 
import numpy as np

In [4]:
df = pd.read_csv(r"C:\Users\nktun\OneDrive\Documents\GitHub\Bank_Marketing\bank (original dataset)\bank-full.csv", delimiter=';')

Understanding the data:  
1 - age (numeric)

   2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                       "blue-collar","self-employed","retired","technician","services") 
   
   3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
   
   4 - education (categorical: "unknown","secondary","primary","tertiary")
   
   5 - default: has credit in default? (binary: "yes","no")
   
   6 - balance: average yearly balance, in euros (numeric) 
   
   7 - housing: has housing loan? (binary: "yes","no")
   
   8 - loan: has personal loan? (binary: "yes","no")
   
   # related with the last contact of the current campaign:
   
   9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 
  
  10 - day: last contact day of the month (numeric)
  
  11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  
  12 - duration: last contact duration, in seconds (numeric)
   
   # other attributes:
  
  13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  
  
  14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  
  15 - previous: number of contacts performed before this campaign and for this client (numeric)
  
  16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

  Output variable (desired target):
  17 - y - has the client subscribed a term deposit? (binary: "yes","no")

# No missing attributes, this is for splitting and training immediately

In [5]:
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [57]:
# Selecting X and y, and then perform train test split
X = df.drop(columns='y')
y = df['y']


In [58]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [63]:
ohe_job = OneHotEncoder(sparse_output=False) # Instantiate a class of OHE
# Job and Marital should be encoded as OHE because of no inherent ranking, but
# Education does have ranking
encoded_job = ohe_job.fit_transform(X[['job']])
# Convert the encoded job back to df format to concat to X
encoded_job_df = pd.DataFrame(encoded_job, columns=ohe_job.get_feature_names_out(['job']))
ohe_marital = OneHotEncoder(sparse_output=False)
encoded_marital = ohe_marital.fit_transform(X[['marital']])
# Convert the encoded job back to df format to concat to X
encoded_marital_df = pd.DataFrame(encoded_marital, columns=ohe_marital.get_feature_names_out(['marital']))

In [65]:
# Concat with the original df
X_encoded = pd.concat([X, encoded_job_df, encoded_marital_df], axis=1)

# Drop the original job column if no longer needed
X_encoded.drop(['job', 'marital'], axis=1, inplace=True)

In [70]:
X_encoded.columns # Ok
X_encoded.head(10)
# Now, we shall map education and replace housing and loan
education_mapping = {'unknown':0, 'primary':1, 'secondary':2, 'tertiary':3}
X_encoded['education'] = X_encoded['education'].map(education_mapping)

In [74]:
# Mapping binary features
binary_mapping = {'yes':1, 'no':0}
X_encoded['default'] = X_encoded['default'].map(binary_mapping)
X_encoded['loan'] = X_encoded['loan'].map(binary_mapping)
X_encoded['housing'] = X_encoded['housing'].map(binary_mapping)

In [86]:
X_encoded[['contact', 'month', 'poutcome']]

,contact,month,poutcome
0,unknown,may,unknown
1,unknown,may,unknown
2,unknown,may,unknown
3,unknown,may,unknown
4,unknown,may,unknown
...,...,...,...
45206,cellular,nov,unknown
45207,cellular,nov,unknown
45208,cellular,nov,success
45209,telephone,nov,unknown


In [ ]:
    # Splitting the data with 25 - 75
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=42,  
                                   test_size=0.25,  
                                   shuffle=True) 
# I use XGBoost so no scaling shall be required
# Encoding is still needed however